# A3C Virtual Morris Water Maze

VMWM unity environment is a play ground to test navigation ability of patient with hippocampus trauma. Test subject is randomly rebornd and is to relocate the platform by using visual cues only after flags (show where the platform is under the water surface) are off.

The A.I. agent is to replicte similar behavior by using state-of-the-art deep learning model. 

![Maze.png](maze.png)

## Import Package

In [1]:
%matplotlib inline
from model import *

## VMWM Setup

In [2]:
max_episode_length = 200
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = 7056 # Observations are greyscale frames of 84 * 84 * 1
a_size = 3 # Agent can move Left, Right, or Fire
load_model = False
model_path = './model'
noisy=True

## VMWM Train

In [3]:
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
#Create a directory to save episode playback gifs to
if not os.path.exists('./frames'):
    os.makedirs('./frames')

with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4)
    master_network = AC_Network(s_size,a_size,'global',None,noisy) # Generate global network
    num_workers = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
    workers = []
    # Create worker classes
    for i in range(1):
        worker = Worker(i,s_size,a_size,trainer,model_path,global_episodes,noisy)
        workers.append(worker)
        worker.start(setting=0)
    saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    # This is where the asynchronous magic happens.
    # Start the "work" process for each worker in a separate thread.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
        t = threading.Thread(target=(worker_work))
        t.start()
        sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)

('127.0.0.1', 52055)
Port being used: 52055
local host--127.0.0.1:52055
Starting worker 0


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:00<00:00, 43.72it/s]


KeyboardInterrupt: 

## My Worker

In [19]:
"""class Worker():

    def __init__(self,name):
        self.name = "worker_" + str(name)
        self.number = name
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1', 0))
        port = sock.getsockname()[1]
        print(sock.getsockname())
        #sock.shutdown(socket.SHUT_WR)
        self.env = VMWMGame(cfg_location,exe_location)
        self.env.reset_cfg()
        self.env.set_trial('Practice - Hills')
        self.env.set_local_host('127.0.0.1', port) # local host IP address & dynamic allocated port 
    
    def start(self):
        self.env.start(setting=1)
        
    def work(self):
        self.env.start_trial()
        sleep(0.5)
        #print('OK')
        for i in range(20):
            #print('In action')
            while(not self.env.is_episode_finished()):
                #cv2.imshow('frame', env.get_screenImage())
                #cv2.waitKey(1)
                self.env.make_action(2,50)
                #print(env.get_reward())
                #print(env.get_score())
            self.env.new_episode()
            
        self.env.end_trial()
        self.env.s.close()"""

'class Worker():\n\n    def __init__(self,name):\n        self.name = "worker_" + str(name)\n        self.number = name\n        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n        sock.bind((\'127.0.0.1\', 0))\n        port = sock.getsockname()[1]\n        print(sock.getsockname())\n        #sock.shutdown(socket.SHUT_WR)\n        self.env = VMWMGame(cfg_location,exe_location)\n        self.env.reset_cfg()\n        self.env.set_trial(\'Practice - Hills\')\n        self.env.set_local_host(\'127.0.0.1\', port) # local host IP address & dynamic allocated port \n    \n    def start(self):\n        self.env.start(setting=1)\n        \n    def work(self):\n        self.env.start_trial()\n        sleep(0.5)\n        #print(\'OK\')\n        for i in range(20):\n            #print(\'In action\')\n            while(not self.env.is_episode_finished()):\n                #cv2.imshow(\'frame\', env.get_screenImage())\n                #cv2.waitKey(1)\n                self.env.make_actio